In [ ]:
#%pdb
%matplotlib inline
import matplotlib.pyplot as plt
import yt
import logging
logging.getLogger('yt').setLevel(logging.ERROR)
import numpy as np
from yt_synchrotron_emissivity import *


In [ ]:
fname = '/home/ychen/data/0only_0529_h1/MHD_Jet_hdf5_plt_cnt_0600'
ds = yt.load(fname, particle_filename=fname.replace('plt_cnt', 'part'))
data = ds.all_data()

In [ ]:
ptype = 'jetp'
proj_axis = 'x'
nu = (150, 'MHz')
#pars = add_synchrotron_emissivity(ds, ptype=ptype, nu=nu)
pars = add_synchrotron_pol_emissivity(ds, ptype=ptype, nu=nu, proj_axis=proj_axis)
fieldi = ('deposit', ('nn_emissivity_i_%s_%%.1f%%s' % ptype) % nu)
fieldq = ('deposit', ('nn_emissivity_q_%s_%%.1f%%s' % ptype) % nu)
fieldu = ('deposit', ('nn_emissivity_u_%s_%%.1f%%s' % ptype) % nu)
proj = yt.ProjectionPlot(ds, proj_axis, [fieldi, fieldq, fieldu], center=[0,0,0], width=((40,'kpc'),(80,'kpc')))
#plot = yt.SlicePlot(ds, proj_axis, [fieldi, fieldq, fieldu], center=[0,0,0], width=((40,'kpc'),(80,'kpc')))

In [ ]:
from yt.utilities.fits_image import FITSProjection
fieldi = ('nn_emissivity_i_%s_%%.1f%%s' % ptype) % nu
#fits_proj = FITSProjection(ds, proj_axis, [fieldi, fieldq, fieldu], center=[0,0,0], width=((40,'kpc'),(80,'kpc')))
width = ds.domain_width[1:]/8
res = ds.domain_dimensions[1:]*ds.refine_by**ds.index.max_level/8
fits_proj = FITSProjection(ds, proj_axis, fieldi, center=[0,0,0], width=width, image_res=res)


In [ ]:
fits_proj.writeto('emissivity_i_600_1.4GHz.fits')

In [ ]:
fieldi = ('deposit', ('nn_emissivity_i_%s_%%.1f%%s' % ptype) % nu)

In [ ]:
frb_I = proj.frb.data[fieldi].v
frb_Q = proj.frb.data[fieldq].v
frb_U = proj.frb.data[fieldu].v
#null = plt.hist(np.log10(arri.flatten()), range=(-15,3), bins=100)

factor = 1
nx = 800/factor
ny = 800/factor

I_bin = frb_I.reshape(nx, factor, ny, factor).sum(3).sum(1)
Q_bin = frb_Q.reshape(nx, factor, ny, factor).sum(3).sum(1)
U_bin = frb_U.reshape(nx, factor, ny, factor).sum(3).sum(1)

psi = 0.5*np.arctan2(U_bin, Q_bin)
frac = np.sqrt(Q_bin**2+U_bin**2)/I_bin
#print len(psi.flatten()[psi.flatten().nonzero()])
#null = plt.hist(np.abs(psi.flatten())[psi.flatten().nonzero()], bins=50)
#plt.xlim(0, 0.5*np.pi)

In [ ]:
fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(131)
null = ax1.hist(frac.flatten()*100, range=(0,80), bins=40)
ax1.set_xlabel('Polarization fraction (%)')
ax1.set_xlim(0, 80)

ax2  = fig.add_subplot(132)
null = ax2.hist(psi.flatten()[psi.flatten().nonzero()], bins=50, range=(-0.5*np.pi, 0.5*np.pi))
x_tick = np.linspace(-0.5, 0.5, 5, endpoint=True)

x_label = [r"$-\pi/2$", r"$-\pi/4$", r"$0$", r"$+\pi/4$", r"$+\pi/2$"]
ax2.set_xlim(-0.5*np.pi, 0.5*np.pi)
ax2.set_xticks(x_tick*np.pi)
ax2.set_xticklabels(x_label)
ax2.set_title(ds.basename + '  %.1f %s' % nu)

ax3 = fig.add_subplot(133)
null = ax3.hist(np.abs(psi.flatten()[psi.flatten().nonzero()]), bins=25, range=(0.0, 0.5*np.pi))
ax3.set_xlim(0.0, 0.5*np.pi)
ax3.set_xticks([x_tick[2:]*np.pi])
ax3.set_xticks(x_tick[2:]*np.pi)
ax3.set_xticklabels(x_label[2:])

In [ ]:

x = np.linspace(proj.frb.bounds[0], proj.frb.bounds[1], nx)
y = np.linspace(proj.frb.bounds[2], proj.frb.bounds[3], ny)
XX, YY = np.meshgrid(x,y)
fig = plt.figure(figsize=(16,10))
plt.scatter(np.log10(I_bin), frac, s=1, lw=0)

In [ ]:
import matplotlib.pyplot as plt
cmap = plt.cm.seismic
plot.set_cmap(field, cmap)
plot.set_zlim(field, -1E1, 1E1)
#plot.zoom(0.5)
plot.show()
FRB = plot.frb

In [ ]:
field = ('deposit', 'nn_emissivity_u_jetp_150.0MHz')
arr = FRB.data[field]
plt.imshow(arr.v, origin='lower', aspect=2, cmap='seismic', vmin=-10, vmax=10)


In [ ]:
plot = yt.ProjectionPlot(ds, 'x', 'density', center=[0,0,0], width=((20,'kpc'),(40,'kpc')))

In [ ]:
field = fieldi
xx0, xx1 = plot.xlim
print xx0, xx1
yy0, yy1 = plot.ylim
print plot.plots

factor = 10.0
nx = plot.plots[field].image._A.shape[0] / factor
ny = plot.plots[field].image._A.shape[1] / factor
X,Y = np.meshgrid(np.linspace(xx0,xx1,nx,endpoint=True),\
                  np.linspace(yy0,yy1,ny,endpoint=True))

#frb_U = X
#frb_Q = Y
#frb_I = np.sqrt(X*X+Y*Y)

psi = 0.5*np.arctan(frb_U[::factor,::factor]/frb_Q[::factor,::factor])
frac = np.sqrt(frb_Q[::factor,::factor]**2+frb_U[::factor,::factor]**2)/frb_I[::factor,::factor]
#null = plt.hist(frac.flatten(), range=(0,5), bins=100)

In [ ]:
plot.annotate_polline(frb_I, frb_Q, frb_U)
plot.set_zlim(fieldi, 1E-2, 1E2)
cmap = plt.cm.get_cmap("kamae_r")
cmap.set_bad((206./256., 221./256., 250./256.))
#cmap = plt.cm.get_cmap("algae")
#cmap.set_bad((80./256., 0.0, 80./256.))
plot.set_cmap(fieldi, cmap)
plot.show()
plot.annotate_clear()

In [ ]:
plot.set_zlim(fieldq, -1E0, 1E0)
plot.set_cmap(fieldq, 'seismic')
plot.set_zlim(fieldu, -1E0, 1E0)
plot.set_cmap(fieldu, 'seismic')

In [ ]:

plot.save()

In [ ]:
p = plot.plots[fieldi]
quiveropts = dict(headlength=code0, headwidth=1, pivot='middle')#, scale=3, 
    #linewidth=.5, units='xy', width=.05, headwidth=1)v
p.axes.cla()
p.axes.quiver(X,Y,frac*np.sin(psi),frac*np.cos(psi), **quiveropts)
#p.show()
plot.show()

In [ ]:
code0 = ds.quan(0, 'code_length')

In [ ]:
plot.zoom(64)
plot.annotate_grids()
plot.annotate_particles((0.5, 'kpc'), col='white', ptype='jetp')
plot.set_zlim(field, 1E-1, 1E3)
plot.set_cmap(field, cmap)
plot.show()

In [ ]:
ptype = 'all'
#B = np.array([data[(ptype, 'particle_magx')], data[(ptype, 'particle_magy')], data[(ptype, 'particle_magz')]])\
B_cut = 1E-20
Bvec = np.array([data[('flash', 'magx')], data[('flash', 'magy')], data[('flash', 'magz')]])\
    *np.sqrt(4.0*np.pi)
print Bvec.shape
#B = B[:,np.nonzero(B2)][:,0,:]
#print B.shape

In [ ]:
B[:,1]
B2 = np.sum(B*B, axis=0)
print B2.shape
flag = B2 > B_cut**2
print flag.shape
print B.shape
print B[:,flag].shape
#print B[:,np.nonzero(B2)].shape
#print B[:,np.nonzero(B2)][:,0,:].shape

In [ ]:
import matplotlib.pyplot as plt
plt.hist(np.log10(B2[B2>0.0]), bins=40)#, range=(2E-8, 1E-7))

#print cross
#print np.sum(cross*cross, axis=1)
#print np.sum(B*B, axis=0)

In [ ]:
for proj_axis in ['x', 'y', 'z']:
    if proj_axis=='x': los = [1.,0.,0.]
    elif proj_axis=='y': los = [0.,1.,0.]
    elif proj_axis=='z': los = [0.,0.,1.]

    cross = np.cross(los, B, axisb=0)
    sina = np.sqrt(np.sum(cross*cross, axis=1)/np.sum(B*B, axis=0))
    #print sina.shape
    #print "sin(alpha)=", sina
    a = np.arcsin(sina)/np.pi*180
    #print 'alpha=', a
    dummy = plt.hist(a, bins=40, alpha=0.5, label=proj_axis)
plt.xlabel('B position angle (deg)')
plt.legend(loc=2)

In [ ]:

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

xs, ys, zs = zip([0,0,0], LOS)
ax.plot(xs, ys, zs)
xs, ys, zs = zip([0,0,0], B0/np.sqrt(np.sum(B0*B0)))
ax.plot(xs, ys, zs)



ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)

In [ ]:
Bvec = np.array([data[('flash', 'magx')], data[('flash', 'magy')], data[('flash', 'magz')]])\
    *np.sqrt(4.0*np.pi)
print Bvec.shape
los = np.array([1.,0.,0.])
print los.shape

In [ ]:
B2 = np.sum(Bvec*Bvec, axis=0)
mask = B2>0.0
Bvec = Bvec[:,mask]
print Bvec.shape
print np.inner(Bvec.transpose(),los)

In [ ]:
inner = np.inner(Bvec.transpose(), los)
print inner.shape
inner.reshape((inner.shape[0], 1))
print inner
print (inner.reshape((-1, 1))*los).shape

#Bproj = Bvec - *los

In [ ]:
Bvec.transpose() - np.inner(Bvec.transpose(), los).reshape((-1,1))*los